In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.signal import find_peaks, peak_prominences
from scipy.stats import entropy

PLOT FUNCTION FOR TIME SERIES

In [ ]:
def pulse_output_plotter(v,Ts,name,t_min,t_max,scatter=False):
    plt.figure(figsize=(10, 8))
    plt.subplot(311)
    plt.plot([Ts[i] for i in range(0,len(Ts)) if Ts[i]> t_min and Ts[i]< t_max],[v[i] for i in range(0,len(Ts)) if Ts[i]> t_min and Ts[i]< t_max], 'b', linewidth=2, label=name)
    plt.xlabel('Time'); plt.ylabel(name);
    plt.legend(loc='upper right')
    plt.grid()
    if scatter is True:
            plt.plot([Ts[i] for i in range(0,len(Ts)) if Ts[i]> t_min and Ts[i]< t_max],[v[i] for i in range(0,len(Ts)) if Ts[i]> t_min and Ts[i]< t_max], 'b', linewidth=2, label=name)

DEFINITION OF THE FUNCTION TO REVERSE INDEX AND VALUES

In [ ]:
def reverser(v):
    rev = {}
    for i in range(0,len(v)):
        rev[str(v[i])] = i
    return rev

DEFINITION OF THE FUNCTION FOR ASSIGN A VALUE FOR THE BURTS DISTANCE OF ONE BURST TIME SERIES

In [ ]:
def one_burst_selector(burst,N):
    if len(burst) == 0:
        return 0
    elif burst[-1] >= 3*N/4:
        return 100000
    else:
        return 0

DEFINITION OF THE FUNCTION TO EVALUATE THE VALUE OF THE POTENTIAL ON THE PEAK POSITION

In [ ]:
def peaks_reveal(v,peak_pos):
    peaks = []
    for i in peak_pos:
        peaks.append(v[int(i)])
    return np.array(peaks)

DEFINITION OF THE FUNCTION TO CONVERT THE TIME SERIES IN BINARY NUMBER AND COUNT THE NUMER OF BINS

In [ ]:
def bin_peaks(v,events,mode = 'binary'):
    v_bin = []
    peak_pos = []
    for Burst in events:
        peak_pos.extend(list(events[Burst]))
    minlength = min(peaks_reveal(v,peak_pos))
    for i in range(0,len(v)):
        if v[i] >= minlength:
            v_bin.append(1)
        else:
            v_bin.append(0)
    v_bin = np.array(v_bin)
    if mode == 'binary':
        return np.bincount(v_bin)
    elif mode == 'bin_plot':
        return v_bin

DEFINITION OF THE FUNCTION FOR THE DETCTION OF THE CHARACTERISTICS OF ONSET AND OFFSET PEAKS

In [ ]:
def Bif_detection(peak_pos,z):
    N = len(peak_pos)
    first_peak = peak_pos[0]
    last_peak = peak_pos[N-1]
    first_peak = peak_pos[0]
    last_peak = peak_pos[N-1]
    first_peak = peak_pos[0]
    last_peak = peak_pos[N-1]
    f_peak_dist = np.floor((peak_pos[1] - peak_pos[0])/2)
    l_peak_dist = np.floor((peak_pos[N-1] - peak_pos[N-2])/2)
    z_left_f_peak = z[int(first_peak-f_peak_dist)]
    
    z_right_f_peak = min([z[i] for i in range(peak_pos[0]-1,peak_pos[1])])
    z_left_l_peak = z[int(last_peak-l_peak_dist)]
    max_prom = max(peak_prominences(z,peak_pos)[0])
    first_prom = peak_prominences(z,peak_pos)[0][0]
    last_prom = peak_prominences(z,peak_pos)[0][-1]
    onset_amp = first_prom /max_prom
    onset_diff = abs(z_left_f_peak) - abs(z_right_f_peak)
    z_right_l_peak = min([z[i] for i in range(peak_pos[N-2] , peak_pos[N-1]+1)])
    offset_amp =  last_prom /max_prom
    offset_diff = z_left_l_peak - z_right_l_peak
    return onset_amp,onset_diff,offset_amp,offset_diff

DEFINITION OF THE FUNCTION FOR THE SEPARATION OF THE BURSTS ON A TIME SERIE

In [ ]:
def events_counter(w,th=0.00012):
    v = reverser(w)
    burst = {}
    n_burst = 0
    burst['0'] = []
    burst['0'].append(w[0])
    for i in range(1,len(w)):
        der = abs(v[str(w[i])]-v[str(w[i-1])])/abs(w[i]-w[i-1])
        if der < th:
            n_burst += 1
            burst[str(n_burst)] = []
        burst[str(n_burst)].append(w[i])
    return burst

DEFINITION OF THE FUNCTION TO DIVIDE THE TIME SERIE IN INTERVALLS

In [ ]:
def intervall_divisor(z,peak_pos,n=3):
    first_peak = peak_pos[0]
    N = peak_pos[-1] - first_peak
    interval = {}
    for i in range(0,n):
        interval[str(i)] = []
        if i == 0:
            for j in range(first_peak,first_peak+int(N/n)):
                if j in peak_pos:
                    interval[str(i)].append(1)
                else:
                    interval[str(i)].append(0)
        else:
            for j in range(first_peak+int(N/(n-i+1)),first_peak+int(N/(n-i))):
                if j in peak_pos:
                    interval[str(i)].append(1)
                else:
                    interval[str(i)].append(0)
        interval[str(i)] = np.array(interval[str(i)])
    return interval

DEFINITION OF THE FUNCTION TO CALCULATE THE ENTROPY OF EACH INTERVALL

In [ ]:
def entropy_intervall(z,peak_pos,n=3,base = 10):
    S_W = {}
    intervall = intervall_divisor(z,peak_pos)
    for i in range(0,n):
        S_W[str(i)] = entropy(intervall[str(i)], base = base)/len(intervall[str(i)])
    return S_W

BIFURCATION CLASSIFIER

In [ ]:
def Bif_classifier(peak_pos,z,th = 6.5,eps=0.1,last = False):
    N = len(peak_pos)
    first_peak = peak_pos[0]
    last_peak = peak_pos[N-1]
    f_peak_dist = np.floor((peak_pos[1] - peak_pos[0])/2)
    l_peak_dist = np.floor((peak_pos[N-1] - peak_pos[N-2])/2)
    z_left_f_peak = z[int(first_peak-f_peak_dist)]
    z_right_f_peak = min([z[i] for i in range(peak_pos[0]-1,peak_pos[1])])
    z_left_l_peak = z[int(last_peak-l_peak_dist)]
    max_prom = max([z[i] for  i in [peak_pos[0],peak_pos[N-1]]])#max(peak_prominences(z,peak_pos)[0])
    first_prom = peak_prominences(z,peak_pos)[0][0]
    last_prom = peak_prominences(z,peak_pos)[0][-1]
    if z[first_peak] /max_prom > eps:
        if abs(z_left_f_peak) - abs(z_right_f_peak) < -th:
            first_bif = 'SNIC'
        else:
            first_bif = 'SN'
    else:
        first_bif = 'Sup H'
    z_right_l_peak = min([z[i] for i in range(peak_pos[N-2] , peak_pos[N-1]+1)])
    if last is True and last_peak >= 3*N/4:
        last_bif = 'Infinity'
        return first_bif,last_bif
    if z[last_peak] /max_prom > eps:
        if z_left_l_peak - z_right_l_peak > th:
            last_bif = 'SNIC'
        else:
            last_bif = 'SH'
    else:
        last_bif = 'Sup H'
    return first_bif,last_bif

DEFINITION OF THE FUNCTION TO CALCULATE ONSET AND OFFSET FEATURES

In [ ]:
def on_off_features(z,events):  
    list_on_amp = []
    list_on_diff = []
    list_off_amp = []
    list_off_diff = []
    for Burst in events:
        bif =  Bif_detection(events[str(Burst)],z)
        list_on_amp.append(bif[0])
        list_on_diff.append(bif[1])
        list_off_amp.append(bif[2])
        list_off_diff.append(bif[3])
    onset_amp = np.mean(np.array(list_on_amp))
    onset_diff = np.mean(np.array(list_on_diff))
    offset_amp = np.mean(np.array(list_off_amp))
    offset_diff = np.mean(np.array(list_off_diff))
    return np.array([onset_amp,onset_diff,offset_amp,offset_diff])

DEFINITION OF THE FUNCTION FOR THE BURSTS FEATURES

In [ ]:
def features_bursts(events,z):
    tau_ave_burst = {}
    tau_var_burst = {}
    tau_slope_burst = {}
    amp_ave_burst = {}
    amp_var_burst = {}
    amp_slope_burst = {}
    for Burst in events:
        tau_ave_burst[str(Burst)] = np.average(events[str(Burst)])
        tau_var_burst[str(Burst)] = np.var(events[str(Burst)])/len(events[str(Burst)])
        slope = (events[str(Burst)][-1]-events[str(Burst)][0])/len(events[str(Burst)])
        if abs(slope) < 0.1:
            tau_slope_burst[str(Burst)] = 0
        elif slope > 0:
            tau_slope_burst[str(Burst)] = 1
        else:
            tau_slope_burst[str(Burst)] = -1
        amp = np.array([z[int(i)] for i in events[str(Burst)]])
        amp_ave_burst[str(Burst)] = np.average(amp)
        amp_var_burst[str(Burst)] = np.var(amp)/len(events[str(Burst)])
        slope = (amp[-1]-amp[0])/(events[str(Burst)][-1]-events[str(Burst)][0])
        if abs(slope) < 0.1:
            amp_slope_burst[str(Burst)] = 0
        elif slope > 0:
            amp_slope_burst[str(Burst)] = 1
        else:
            amp_slope_burst[str(Burst)] = -1
    return tau_ave_burst, tau_var_burst, amp_ave_burst, amp_var_burst, tau_slope_burst, amp_slope_burst

DEFINITION OF THE FUNCTION FOR THE TIME SERIES FEATURES

In [ ]:
def features_series(events,z,peak_pos,mode = 'all'):
    f = features_bursts(events,z)
    ave_on_serie_amp = 0
    var_on_serie_amp = 0
    ave_on_serie_tau = 0
    var_on_serie_tau = 0
    Tau = []
    time = []
    binary_arr = bin_peaks(z,events)
    en = entropy(binary_arr, base = 10)/len(z)
    S = en
    if len(events) == 1:
        Slow_ave_ts = len(z)
        ISI_ave_serie = one_burst_selector(burst=events['0'],N = len(z))
        ISI_var_serie = 0
    else:
        for Burst in events:
            if int(Burst) < len(events)-1:
                Tau.append(events[str(int(Burst)+1)][0]-events[str(Burst)][-1])
                time.append(events[str(int(Burst)+1)][-1]-events[str(Burst)][-1])
        Tau = np.array(Tau)
        time = np.array(time)
        Slow_ave_ts = np.average(time)
        ISI_ave_serie = np.average(Tau)
        ISI_var_serie = np.var(Tau)
    binary_arr = bin_peaks(z,events)
    en = entropy(binary_arr, base = 10)/len(z)
    S = en
    Num_of_spikes = 0
    for Burst in events:
        ave_on_serie_tau += f[0][str(Burst)]/(len(events))
        var_on_serie_tau += f[1][str(Burst)]/(len(events))
        ave_on_serie_amp += f[2][str(Burst)]/(len(events))
        var_on_serie_amp += f[3][str(Burst)]/(len(events))
        Num_of_spikes += len(events[str(Burst)])/len(z)
        Num_burst = len(events)/len(z)

    ent_window = entropy_intervall(z,peak_pos)
    S_W_1 = ent_window['0']
    S_W_2 = ent_window['1']
    S_W_3 = ent_window['2']
    S_comb = S*4218 + S_W_1 + S_W_2 + S_W_3
    on_off = on_off_features(z,events)
    onset_amp = on_off[0]
    onset_diff = on_off[1]
    offset_amp = on_off[2]
    offset_diff = on_off[3]
    off_diff_amp = np.log2(offset_diff+6.89*offset_amp)
    on_off_amp = np.log2(onset_amp + offset_amp)
    on_off_diff = onset_diff - 2*offset_diff
    off_diff_ave_amp = offset_diff + 0.368*ave_on_serie_amp
    on_amp_ave_amp = onset_amp + 0.0538*ave_on_serie_amp
    off_amp_ave_amp = offset_amp + 0.0538*ave_on_serie_amp
    features = {'ave_on_serie_amp':ave_on_serie_amp,'log2(var_on_serie_amp)':np.log2(var_on_serie_amp),'log2(S_comb)':np.log2(S_comb),'off_diff_amp': off_diff_amp,'on_off_amp':on_off_amp,
                    'on_off_diff':on_off_diff,'off_diff_ave_amp':off_diff_ave_amp,'on_amp_ave_amp':on_amp_ave_amp,'off_amp_ave_amp':off_amp_ave_amp,
                    'ave_on_serie_tau':ave_on_serie_tau,'var_on_serie_tau':var_on_serie_tau,'Num_burst':Num_burst,'Num_of_spikes':Num_of_spikes,
                    'ISI_ave_serie':ISI_ave_serie,'ISI_var_serie':ISI_var_serie,'Slow_ave_ts':Slow_ave_ts}
    #features = np.array([ave_on_serie_amp,np.log2(var_on_serie_amp), np.log2(S_comb),off_diff_amp,on_off_amp,on_off_diff,off_diff_ave_amp,on_amp_ave_amp,off_amp_ave_amp])
    if mode == 'all':
        return features
    else:
        selected_features = {}
        for name in mode:
            selected_features[name] = features[name]
        return selected_features

DEFINITION OF THE FUNCTION TO CALCULATE THE PEAKS POSITION

In [2]:
def Peak_pos(z):
    return find_peaks(z,height= -20)[0]